In [34]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [35]:
# Kiểm tra PyTorch và GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("Warning: No GPU found. Running on CPU.")

PyTorch version: 2.6.0+cu118
CUDA available: True
Number of GPUs: 1
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU


In [36]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [37]:
# Bước 1: Tải dữ liệu từ SQLite
def load_data_from_sqlite():
    conn = sqlite3.connect('historical_data.db')
    query = "SELECT * FROM historical_data ORDER BY item_id, store_id, date"
    data = pd.read_sql_query(query, conn)
    conn.close()
    
    data['date'] = pd.to_datetime(data['date'])
    
    print("Data loaded from SQLite successfully!")
    print(f"Data shape: {data.shape}")
    
    print("Checking for NaN in features...")
    print(data.isna().sum())
    
    return data

In [38]:
# Bước 2: Dataset và DataLoader cho PyTorch
class SalesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def prepare_lstm_data(data, features, sequence_length=28):
    X, y = [], []
    unique_items = data[['item_id', 'store_id']].drop_duplicates().values
    
    for item_id, store_id in unique_items:
        item_data = data[
            (data['item_id'] == item_id) & (data['store_id'] == store_id)
        ].sort_values('date')
        
        item_features = item_data[features].values
        
        for i in range(len(item_data) - sequence_length):
            X.append(item_features[i:i + sequence_length])
            y.append(item_features[i + sequence_length, 0])  # sales là mục tiêu
    
    X = np.array(X)  # Shape: (samples, sequence_length, num_features)
    y = np.array(y)  # Shape: (samples,)
    
    # Chia dữ liệu thành tập huấn luyện và kiểm tra (70-30)
    train_size = int(0.7 * len(X))
    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]
    
    # Tạo Dataset và DataLoader
    train_dataset = SalesDataset(X_train, y_train)
    val_dataset = SalesDataset(X_val, y_val)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Giảm batch size xuống 32
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    print("LSTM data prepared successfully!")
    print(f"X_train shape: {X_train.shape}")
    print(f"X_val shape: {X_val.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"y_val shape: {y_val.shape}")
    
    return train_loader, val_loader, X_val, y_val

In [ ]:
# Bước 3: Xây dựng mô hình LSTM với PyTorch
class LSTMModel(nn.Module):
    def __init__(self, input_size=12, hidden_size=128, num_layers=3, dropout=0.3):  
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_size, 32)  # Giảm số units
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [40]:
# Bước 4: Huấn luyện mô hình
def train_model(model, train_loader, val_loader, num_epochs=20):  # Số epoch đã giảm xuống 20
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=2, min_lr=1e-5)
    
    best_val_loss = float('inf')
    patience = 5
    counter = 0
    
    for epoch in range(num_epochs):
        # Huấn luyện
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            outputs, y_batch = outputs.squeeze(), y_batch.squeeze()
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X_batch.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        # Đánh giá
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                outputs, y_batch = outputs.squeeze(), y_batch.squeeze()
                loss = criterion(outputs, y_batch)
                val_loss += loss.item() * X_batch.size(0)
        
        val_loss /= len(val_loader.dataset)
        
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        
        # Early stopping
        scheduler.step(val_loss)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), 'lstm_model.pth')
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered.")
                break
    
    return model

In [41]:
# Bước 5: Đánh giá mô hình (Chia nhỏ dữ liệu thành batch)
def evaluate_model(model, X_val, y_val, batch_size=32):
    model.eval()
    dataset = SalesDataset(X_val, y_val)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    y_pred_list = []
    with torch.no_grad():
        for X_batch, _ in data_loader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            outputs = outputs.squeeze().cpu().numpy()
            y_pred_list.append(outputs)
    
    y_pred = np.concatenate(y_pred_list, axis=0)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mae = mean_absolute_error(y_val, y_pred)
    
    print(f"Validation RMSE: {rmse:.4f}")
    print(f"Validation MAE: {mae:.4f}")
    
    # Giải phóng bộ nhớ
    torch.cuda.empty_cache()
    
    return y_pred, rmse, mae

In [42]:
# Hàm chính
def main():
    # Giải phóng bộ nhớ trước khi bắt đầu
    torch.cuda.empty_cache()
    
    data = load_data_from_sqlite()
    
    features = ['sales', 'sell_price', 'day_of_week', 'snap_CA', 'is_holiday', 'month', 'day_of_month',
                'sales_lag_7', 'sales_lag_14', 'sales_lag_28', 'sales_roll_mean_7', 'sales_roll_mean_14']
    
    train_loader, val_loader, X_val, y_val = prepare_lstm_data(data, features, sequence_length=28)
    
    model = LSTMModel(input_size=len(features)).to(device)
    
    model = train_model(model, train_loader, val_loader)
    
    y_pred, rmse, mae = evaluate_model(model, X_val, y_val)
    
    print("Model training completed and saved.")
    
    return model, y_pred, rmse, mae

if __name__ == "__main__":
    model, y_pred, rmse, mae = main()

Data loaded from SQLite successfully!
Data shape: (304920, 15)
Checking for NaN in features...
item_id               0
store_id              0
date                  0
sales                 0
sell_price            0
day_of_week           0
snap_CA               0
is_holiday            0
month                 0
day_of_month          0
sales_lag_7           0
sales_lag_14          0
sales_lag_28          0
sales_roll_mean_7     0
sales_roll_mean_14    0
dtype: int64
LSTM data prepared successfully!
X_train shape: (208504, 28, 12)
X_val shape: (89360, 28, 12)
y_train shape: (208504,)
y_val shape: (89360,)
Epoch 1/20, Train Loss: 0.0003, Val Loss: 0.0001
Epoch 2/20, Train Loss: 0.0003, Val Loss: 0.0001
Epoch 3/20, Train Loss: 0.0003, Val Loss: 0.0001
Epoch 4/20, Train Loss: 0.0003, Val Loss: 0.0001
Epoch 5/20, Train Loss: 0.0003, Val Loss: 0.0001
Epoch 6/20, Train Loss: 0.0003, Val Loss: 0.0001
Epoch 7/20, Train Loss: 0.0002, Val Loss: 0.0001
Epoch 8/20, Train Loss: 0.0002, Val Loss: 0.0001